In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import PredefinedSplit, GridSearchCV

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/kdr/train_1.csv')
test = pd.read_csv('../../../../data/hi/kdr/test_1.csv')

train

,Unnamed: 0,smiles,value
0,0,Brc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,True
1,1064,CCc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,False
2,1065,CCc1ccc(-c2nc3ccc(Nc4ncnc5ccccc45)cc3[nH]2)cc1,False
3,3722,COc1ccccc1-c1nc2ccc(Nc3ccnc4ccccc34)cc2[nH]1,False
4,4159,Cc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,False
...,...,...,...
495,454,CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2cccc(NC(=O)CN)c...,False
496,1850,COC(=O)c1cn2ncnc(Oc3ccc4[nH]c(C)cc4c3F)c2c1C,True
497,4120,Cc1cc2c(F)c(Oc3ncnn4cc(OCCCNS(C)(=O)=O)c(C)c34...,True
498,2979,COc1cc2c(Oc3ccc(N/C=C4\C(=O)NC(=O)N(c5ccc(C)cc...,True


In [4]:
from scipy.spatial.distance import jaccard


def run_knn_gridsearch_tanimoto(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
        'n_neighbors': [3, 5, 7, 10, 12, 15],
        'weights': ['uniform', 'distance'],
    }
    knn = KNeighborsClassifier(metric=jaccard)

    grid_search = GridSearchCV(knn, params, cv=pds, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'], metric=jaccard)
    knn.fit(train_fps, train['value'])

    train_preds = knn.predict_proba(train_fps)[:, 1]
    train_metrics = get_hi_metrics(train, train_preds)

    test_preds = knn.predict_proba(test_fps)[:, 1]
    test_metrics = get_hi_metrics(test, test_preds)
    return train_metrics, test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

train_metrics, test_metrics = run_knn_gridsearch_tanimoto(train_maccs_fps, test_maccs_fps)
print(train_metrics)
print(test_metrics)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[CV 1/1] END ....n_neighbors=3, weights=uniform;, score=0.550 total time=  13.0s
[CV 1/1] END ...n_neighbors=3, weights=distance;, score=0.547 total time=  12.2s
[CV 1/1] END ....n_neighbors=5, weights=uniform;, score=0.544 total time=  12.4s
[CV 1/1] END ...n_neighbors=5, weights=distance;, score=0.537 total time=  12.3s
[CV 1/1] END ....n_neighbors=7, weights=uniform;, score=0.542 total time=  11.7s
[CV 1/1] END ...n_neighbors=7, weights=distance;, score=0.536 total time=  11.9s
[CV 1/1] END ...n_neighbors=10, weights=uniform;, score=0.538 total time=  12.5s
[CV 1/1] END ..n_neighbors=10, weights=distance;, score=0.532 total time=  12.7s
[CV 1/1] END ...n_neighbors=12, weights=uniform;, score=0.533 total time=  11.8s
[CV 1/1] END ..n_neighbors=12, weights=distance;, score=0.526 total time=   9.6s
[CV 1/1] END ...n_neighbors=15, weights=uniform;, score=0.528 total time=   9.7s
[CV 1/1] END ..n_neighbors=15, weights=distance;

# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_morgan_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_morgan_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

    knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric=jaccard)
    knn.fit(train_morgan_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = knn.predict_proba(train_morgan_fps)[:, 1]

    test_result = test.copy()
    test_result['preds'] = knn.predict_proba(test_morgan_fps)[:, 1]

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/kdr/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/kdr/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/kdr/knn_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/kdr/knn_maccs/test_{i}.csv')
